In [170]:
import numpy as np
from osgeo import gdal, osr
from scipy.spatial import Delaunay
from matplotlib.patches import Polygon
import matplotlib.pyplot as plt
from matplotlib import transforms

plt.style.use('default')

In [171]:
in_dem = r"..\Data\N46E009.hgt"
dem_ds = gdal.Open(in_dem)
prj = dem_ds.GetProjection()
srs = osr.SpatialReference(wkt=prj)
dem_band = dem_ds.GetRasterBand(1)
dem_nodata = dem_band.GetNoDataValue()
scale = 111120 if srs.IsGeographic() else 1.0

In [172]:
# Calculate Hillshade
hillsha = gdal.DEMProcessing('', dem_ds, 'hillshade', multiDirectional=True, computeEdges=False,
                                      format='MEM', scale = scale)
hillsha = hillsha.ReadAsArray()

In [173]:
# Generate some random points
N = 50000
x = np.random.uniform(0.0, 3600.0, size=N)
y = np.random.uniform(0.0, 3600.0, size=N)
pts = np.c_[x, y]

In [174]:
# Triangulate points
tri = Delaunay(pts)

In [175]:
# This is slow... 
triangles_colors = {}
rows, cols = hillsha.shape
for i in range(0, rows):
    for j in range(0, cols):
        # Find the triangle
        idx = tri.find_simplex((i, j))
        
        # Point outside the triangulation
        if int(idx) == -1:
            continue
        else:
            # Get Hillshade value
            color_i = hillsha[i, j]
            if int(idx) in triangles_colors:
                triangles_colors[int(idx)].append(color_i)
            else:
                triangles_colors[int(idx)] = [color_i]

In [ ]:
# Plot the result - FIX: rotate image by -90
fig, ax = plt.subplots(figsize=(10,10))
for idx, values in triangles_colors.items():
    triangle = tri.points[tri.simplices[idx]]
    mean_hs = sum(values)/len(values)
    path = Polygon(triangle, 
                   facecolor = (mean_hs/255, mean_hs/255, mean_hs/255))
    ax.add_patch(path)
ax.set_xlim(0, 3600.0)
ax.set_ylim(0, 3600.0)
plt.axis("off")
plt.tight_layout()
plt.savefig("03_Polygons.png", dpi=300)
plt.show()